# **Boosting Wav2Vec2 with n-grams in 🤗 Transformers**

**Wav2Vec2** is a popular pre-trained model for speech recognition. Released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Meta AI Research, the novel architecture catalyzed progress in self-supervised pretraining for speech recognition, *e.g.* [*G. Ng et al.*, 2021](https://arxiv.org/pdf/2104.03416.pdf), [*Chen et al*, 2021](https://arxiv.org/abs/2110.13900), [*Hsu et al.*, 2021](https://arxiv.org/abs/2106.07447) and [*Babu et al.*, 2021](https://arxiv.org/abs/2111.09296). On the Hugging Face Hub, Wav2Vec2's most popular pre-trained checkpoint currently amounts to over [**250,000** monthly downloads](https://huggingface.co/facebook/wav2vec2-base-960h).

Using Connectionist Temporal Classification (CTC), pre-trained Wav2Vec2-like checkpoints are extremely easy to fine-tune on downstream speech recognition tasks.
In a nutshell, fine-tuning pre-trained Wav2Vec2 checkpoints works as follows: 

A single randomly initialized linear layer is stacked on top of the pre-trained checkpoint and trained to classify raw audio input to a sequence of letters. It does so by:

1.  extracting audio representations from the raw audio (using CNN layers),
2. processing the sequence of audio representations with a stack of transformer layers, and,
3. classifying the processed audio representations into a sequence of output letters.

Previously audio classification models required an additional language model (LM) and a dictionary to transform the sequence of classified audio frames to a coherent transcription.
Wav2Vec2's architecture is based on transformer layers, thus giving each processed audio representation context 
from all other audio representations. In addition, 
Wav2Vec2 leverages the [CTC algorithm](https://distill.pub/2017/ctc/) for fine-tuning, which solves the problem of alignment between a varying "input audio length"-to-"output text length" ratio.

Having contextualized audio classifications and no alignment problems, Wav2Vec2 does not require 
an external language model or dictionary to yield acceptable audio transcriptions.

As can be seen in Appendix C of the [official paper](https://arxiv.org/abs/2006.11477), Wav2Vec2 gives impressive downstream performances on [LibriSpeech]((https://huggingface.co/datasets/librispeech_asr)) without using a language model at all. However, from the appendix, it also becomes clear that using Wav2Vec2 in combination with a language model can yield a significant improvement, especially when the model was trained on only 10 minutes of transcribed audio.

Until recently, the 🤗 Transformers library did not offer a simple user interface to decode audio files with a fine-tuned Wav2Vec2 **and** a language model. This has thankfully changed. 🤗 Transformers now offers an easy-to-use integration with *Kensho Technologies'* [pyctcdecode library](https://github.com/kensho-technologies/pyctcdecode). This blog post is a step-by-step **technical** guide to explain how one can create an **n-gram** language model and combine it with an existing fine-tuned Wav2Vec2 checkpoint using 🤗 Datasets and 🤗 Transformers.

We start by:

1. How does decoding audio with an LM differ from decoding audio without an LM?
2. How to get suitable data for a language model?
3. How to build an *n-gram* with KenLM?
4. How to combine the *n-gram* with a fine-tuned Wav2Vec2 checkpoint?

For a deep dive into how Wav2Vec2 functions - which is not necessary for this blog post - the reader is advised to consult the following material:

- [wav2vec 2.0: A Framework for Self-Supervised Learning of Speech Representations](https://arxiv.org/abs/2006.11477)
- [Fine-Tune Wav2Vec2 for English ASR with 🤗 Transformers](https://huggingface.co/blog/fine-tune-wav2vec2-english)
- [An Illustrated Tour of Wav2vec 2.0](https://jonathanbgn.com/2021/09/30/illustrated-wav2vec-2.html)

## **1. Decoding audio data with Wav2Vec2 and a language model**

As shown in 🤗 Transformers [exemple docs of Wav2Vec2](https://huggingface.co/docs/transformers/master/en/model_doc/wav2vec2#transformers.Wav2Vec2ForCTC), audio can be transcribed as follows.

First, we install `datasets` and `transformers`.



In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode
!pip install git+https://github.com/huggingface/transformers.git
!pip install datasets torchaudio

  Using cached https://github.com/kpu/kenlm/archive/master.zip
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-cp7v8m3a
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-cp7v8m3a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.16.0.dev0-py3-none-any.whl size=3510301 sha256=ce718a5c1c88e23a14e34e5499854f1a8958fec906af370198ce1139f623e607
  Stored in directory: /tmp/pip-ephem-wheel-cache-ofmru5mv/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.15.0
    Uninstalling transformers-4.15.0:
      Successfully uninstalled transformers-4.15.0


Let's load a small excerpt of the [Librispeech dataset](https://huggingface.co/datasets/librispeech_asr) to demonstrate Wav2Vec2's speech transcription capabilities.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "mozilla-foundation/common_voice_7_0", "as", data_dir="./cv-corpus-7.0-2021-07-21", use_auth_token=True, split="test")
dataset.save_to_disk("./cv-corpus-7.0-2021-07-21")
dataset

Using custom data configuration as-08527a4ca7b6e8cd
Reusing dataset common_voice (/root/.cache/huggingface/datasets/mozilla-foundation___common_voice/as-08527a4ca7b6e8cd/7.0.0/33e08856cfa0d0665e837bcad73ffd920a0bc713ce8c5fffb55dbdf1c084d5ba)


Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 281
})

In [ ]:
!ls -ltr

total 8
drwxr-xr-x 1 root root 4096 Jan  7 14:33 sample_data
drwxr-xr-x 2 root root 4096 Jan 23 09:15 cv-corpus-7.0-2021-07-21


We can pick one of the 73 audio samples and listen to it.

In [ ]:
import IPython.display as ipd

index = 3
audio_sample = dataset[index]
print(audio_sample["sentence"].lower())
ipd.Audio(data=audio_sample["audio"]["array"], autoplay=True, rate=audio_sample["audio"]["sampling_rate"])

"কানায়ে ৰজাৰ আগত আঁঠু লৈ ক'লে।"


Having chosen a data sample, we now load the fine-tuned model and processor.

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

repo_name = 'infinitejoy/wav2vec2-large-xls-r-300m-assamese'

processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)

Downloading:   0%|          | 0.00/214 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/304 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/729 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Next, we process the data

In [ ]:
inputs = processor(audio_sample["audio"]["array"], sampling_rate=16_000, return_tensors="pt")

In [ ]:
inputs = processor_with_lm(audio_sample["audio"]["array"], sampling_rate=16_000, return_tensors="pt")

In [ ]:
processor_with_lm

forward it to the model

In [ ]:
import torch

with torch.no_grad():
  logits = model(**inputs).logits

and decode it

In [ ]:
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor_with_lm.batch_decode(predicted_ids)

transcription[0].lower()

ValueError: ignored

In [ ]:
import torchaudio

# processor = Wav2Vec2Processor.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")
# model = Wav2Vec2ForCTC.from_pretrained("airesearch/wav2vec2-large-xlsr-53-th")

#function to resample to 16_000
def speech_file_to_array_fn(batch, 
                            text_col="sentence", 
                            fname_col="path",
                            resampling_to=16000):
    print(batch)
    speech_array, sampling_rate = torchaudio.load(batch[fname_col])
    resampler=torchaudio.transforms.Resample(sampling_rate, resampling_to)
    batch["speech"] = resampler(speech_array)[0].numpy()
    batch["sampling_rate"] = resampling_to
    batch["target_text"] = batch[text_col]
    return batch

#get 2 examples as sample input
test_dataset = dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)

#infer
with torch.no_grad():
    logits = model(inputs.input_values,).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["sentence"][:2])

  0%|          | 0/281 [00:00<?, ?ex/s]

{'client_id': 'd17ac41afb58f081fa11de8768837b0c19330492026207574fb05784e8b9e9451de66cbb06dcf84562501c01176c2539fed0961791734ea310971af3b138ab70', 'path': 'cv-corpus-7.0-2021-07-21/as/clips/common_voice_as_20870771.mp3', 'audio': {'bytes': b'ID3\x04\x00\x00\x00\x00\x00#TSSE\x00\x00\x00\x0f\x00\x00\x03Lavf56.40.101\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xfbT\xc4\x00\x03\xc0\x00\x01\xa4\x00\x00\x00 \x03\x80@\x00 \x00\x04\x14\x01\x04\x80@(\x1e\xc4x\xe7\x16\x19\x9f\xaf_\xec,\xe9\x9c\xe9\x9a,3\xfdOj\x9ey\xef\x99\x9e\xffW\xff\xfb*\xa1\xa7\x0e\x8d\x88\x8b\x83\x01\x80\x9c\x1e\x0f\x94\x1d\x1e"\\^$\r\xcc_\xff\xfd\x1a\xd1E\x1d/\x936\x8a\'\x9dy@\x96(\x10Cp\x8ct\'\x9c\x1d\xa4Jt\xa4\xd0\xcc\xe0X\xc4\x1c\xe3\x1f\';\x90\xc0\x06\xe2\xb4M\x0b\xc6\x14\x8c\x86\xf9/l\x99\xe3Z\xbd^\xcf"qQ\x01F\x90\x13De\xcf\xf1\xebZp_\x10\xb2N\x88\'\n\x06\x05\x02,L\x1a\x1f\xa7\xd4q\xee\xd8\x00_\x88\x88\x87\xb2\x04\x1e\xec\x1cx \xc6?A\x02\x1d\xdd\xca\xbb\xff\x0b\xff\xfbT\xc4U\x03\xcb\x15&l&\x1d\x93\xc9\xf9\xb1]@\xf3\

RuntimeError: ignored

In [ ]:
!ls -ltr ~/.cache/huggingface/datasets/mozilla-foundation___common_voice/as-08527a4ca7b6e8cd/7.0.0/33e08856cfa0d0665e837bcad73ffd920a0bc713ce8c5fffb55dbdf1c084d5ba

total 35520
-rw-r--r-- 1 root root 20119920 Jan 21 09:45 common_voice-train.arrow
-rw-r--r-- 1 root root 11121096 Jan 21 09:45 common_voice-test.arrow
-rw-r--r-- 1 root root  3741504 Jan 21 09:45 common_voice-validation.arrow
-rw-r--r-- 1 root root     1496 Jan 21 09:45 common_voice-other.arrow
-rw-r--r-- 1 root root  1365840 Jan 21 09:45 common_voice-invalidated.arrow
-rw-r--r-- 1 root root       50 Jan 21 09:45 LICENSE
-rw-r--r-- 1 root root     3821 Jan 21 09:45 dataset_info.json


Comparing the transcription to the target transcription above, we can see that some words *sound* correct, but are not *spelled* correctly, *e.g.*:

- *christmaus* vs. *christmas*
- *rose* vs. *roast*
- *simalyis* vs. *similes*

Let's see whether combining Wav2Vec2 with an ***n-gram*** lnguage model can help here.

First, we need to install `pyctcdecode` and `kenlm`.

In [ ]:
!pip install https://github.com/kpu/kenlm/archive/master.zip pyctcdecode

     / 541 kB 2.3 MB/s
     |████████████████████████████████| 43 kB 1.2 MB/s 
     |████████████████████████████████| 376 kB 9.1 MB/s 
  Created wheel for kenlm: filename=kenlm-0.0.0-cp37-cp37m-linux_x86_64.whl size=2335801 sha256=34f5918b31ff0eb46feb619399879736f4aa81eb046692d6790971f8f9d8be9d
  Stored in directory: /tmp/pip-ephem-wheel-cache-vpasws9d/wheels/3d/aa/02/7b4a2eab5d7a2a9391bd9680dbad6270808a147bc3b7047e4e
  Created wheel for pygtrie: filename=pygtrie-2.4.2-py3-none-any.whl size=19063 sha256=f23afae85899200c0cb283b47f12b71ea8e3909ca50e8708ff66ae9c79d33d12
  Stored in directory: /root/.cache/pip/wheels/d3/f8/ba/1d828b1603ea422686eb694253a43cb3a5901ea4696c1e0603
Successfully built kenlm pygtrie


For demonstration purposes, we have prepared a new model repository [patrickvonplaten/wav2vec2-base-100h-with-lm](https://huggingface.co/patrickvonplaten/wav2vec2-base-100h-with-lm) which contains the same Wav2Vec2 checkpoint but has an additional **4-gram** language model for English.

Instead of using `Wav2Vec2Processor`, this time we use `Wav2Vec2ProcessorWithLM` to load the **4-gram** model in addition to the feature extractor and tokenizer.

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

# processor = Wav2Vec2ProcessorWithLM.from_pretrained("patrickvonplaten/wav2vec2-base-100h-with-lm")
processor = Wav2Vec2ProcessorWithLM.from_pretrained(repo_name)

Downloading:   0%|          | 0.00/703 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.98G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/78.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.7M [00:00<?, ?B/s]

In constrast to decoding the audio without language model, the processor now directly receives the model's output `logits` instead of the `argmax(logits)` (called `predicted_ids`) above. The reason is that when decoding with a language model, at each time step, the processor takes the probabilities of all possible output characters into account. Let's take a look at the dimension of the `logits` output.

In [ ]:
logits.shape

torch.Size([1, 503, 69])

We can see that the `logits` correspond to a sequence of 624 vectors each having 32 entries. Each of the 32 entries thereby stands for the logit probability of one of the 32 possible output characters of the model:

In [ ]:
" ".join(sorted(processor.tokenizer.get_vocab()))

'</s> <s> [PAD] [UNK] | । ঁ ং ঃ অ আ ই ঈ উ এ ঐ ও ঔ ক খ গ ঘ ঙ চ ছ জ ঝ ঞ ট ঠ ড ঢ ণ ত থ দ ধ ন প ফ ব ভ ম য র ল শ ষ স হ ় া ি ী ু ূ ৃ ে ৈ ো ৌ ্ ৎ ড় ঢ় য় ৰ ৱ ’'

In [ ]:
" ".join(sorted(processor_with_lm.tokenizer.get_vocab()))

'</s> <s> [PAD] [UNK] | । ঁ ং ঃ অ আ ই ঈ উ এ ঐ ও ঔ ক খ গ ঘ ঙ চ ছ জ ঝ ঞ ট ঠ ড ঢ ণ ত থ দ ধ ন প ফ ব ভ ম য র ল শ ষ স হ ় া ি ী ু ূ ৃ ে ৈ ো ৌ ্ ৎ ড় ঢ় য় ৰ ৱ ’'

Intuitively, one can understand the decoding process of `Wav2Vec2ProcessorWithLM` as applying beam search through a matrix of size 624 $\times$ 32 probabilities while leveraging the probabilities of the next letters as given by the *n-gram* language model.

OK, let's run the decoding step again. `pyctcdecode` language model decoder does not automatically convert `torch` tensors to `numpy` so we'll have to convert them ourselves before.

In [ ]:
transcription = processor.batch_decode(logits.numpy()).text
transcription[0].lower()

''

In [ ]:
transcription = processor_with_lm.batch_decode(logits.numpy()).text
transcription[0].lower()

''

Cool! Recalling the words `facebook/wav2vec2-base-100h` without a language model transcribed incorrectly previously, *e.g.*,

> - *christmaus* vs. *christmas*
- *rose* vs. *roast*
- *simalyis* vs. *similes*

we can take another look at the transcription of `facebook/wav2vec2-base-100h` **with** a 4-gram language model. 2 out of 3 errors are corrected; *christmas* and *similes* have been correctly transcribed.

Interestingly, the incorrect transcription of *rose* persists. However, this should not surprise us very much. Decoding audio without a language model is much more prone to yield spelling mistakes, such as *christmaus* or *similes* (those words don't exist in the English language as far as I know). This is because the speech recognition system almost solely bases its prediction on the acoustic input it was given and not really on the language modeling context of previous and successive predicted letters ${}^1$. 
If on the other hand, we add a language model, we can be fairly sure that the speech recognition system will heavily reduce spelling errors since a well-trained *n-gram* model will surely not predict a word that has spelling errors. But the word *rose* is a valid English word and therefore the 4-gram will predict this word with a probability that is not insignificant. 

The language model on its own most likely does favor the correct word *roast* since the word sequence *roast beef* is much more common in English than *rose beef*. Because the final transcription is derived from a weighted combination of `facebook/wav2vec2-base-100h` output probabilities and those of the *n-gram* language model, it is quite common to see incorrectly transcribed words such as *rose*.

For more information on how you can tweak different parameters when decoding with `Wav2Vec2ProcessorWithLM`, please take a look at the official documentation [here](https://huggingface.co/docs/transformers/master/en/model_doc/wav2vec2#transformers.Wav2Vec2ProcessorWithLM.batch_decode).

---
${}^1$ Some research shows that a model such as `facebook/wav2vec2-base-100h` - when sufficiently large and trained on enough data - can learn language modeling dependencies between intermediate audio representations similar to a language model.


Great, now that you have seen the advantages adding an *n-gram* language model can bring, let's dive into how to create an *n-gram* and `Wav2Vec2ProcessorWithLM` from scratch.

## **2. Getting data for your language model**

A language model that is useful for a speech recognition system should support the acoustic model, *e.g.* Wav2Vec2, in predicting the next word (or token, letter) and therefore model the following distribution:

$\mathbf{P}(w_n | \mathbf{w}_0^{t-1})$ with $w_n$ being the next word and $\mathbf{w}_0^{t-1}$ being the sequence of all previous words since the beginning of the utterance. Simply said, the language model should be good at predicting the next word given all previously transcribed words regardless of the audio input given to the speech recognition system.

As always a language model is only as good as the data it is trained on. In the case of speech recognition, we should therefore ask ourselves for what kind of data, the speech recognition will be used for: *conversations*, *audiobooks*, *movies*, *speeches*, *, etc*, ...?

The language model should be good at modeling language that corresponds to the 
target transcriptions of the speech recognition system. 
For demonstration purposes, we assume here that we have fine-tuned a pre-trained [`facebook/wav2vec2-xls-r-300m`](https://huggingface.co/facebook/wav2vec2-xls-r-300m) on [Common Voice 7](https://huggingface.co/datasets/mozilla-foundation/common_voice_7_0) in Swedish. The fine-tuned checkpoint can 
be found [here](https://huggingface.co/hf-test/xls-r-300m-sv).
Common Voice 7 is a relatively crowd-sourced read-out audio dataset and we will evaluate the model on its test data.

Let's now look for suitable text data on the Hugging Face Hub. We search all datasets for those [that contain Swedish data](https://huggingface.co/datasets?languages=languages:sv&sort=downloads). 
Browsing a bit through the datasets, we are looking for a dataset that is similar to Common Voice's read-out audio data. The obvious choices of [oscar](https://huggingface.co/datasets/oscar) and [mc4](https://huggingface.co/datasets/mc4) might not be the most suitable here because they:

- are generated from crawling the web, which might not be very clean and correspond well to spoken language
- require a lot of pre-processing
- are very large which is not ideal for demonstration purposes here 😉

A dataset that seems sensible here and which is relatively clean and easy to pre-process is [europarl_bilingual](https://huggingface.co/datasets/europarl_bilingual) as it's a dataset that is based on discussions and talks of the European parliament. It should therefore be relatively clean and correspond well to read-out audio data. The dataset is originally designed for machine translation and can therefore only be accessed in translation pairs. We will only extract the text of the target language, Swedish (`sv`), from the *English-to-Swedish* translations.

In [ ]:
target_lang="as"  # change to your target lang

Let's download the data.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("europarl_bilingual", lang1="en", lang2=target_lang, split="train")
dataset

Downloading:   0%|          | 0.00/2.60k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.6k [00:00<?, ?B/s]

Using custom data configuration en-as-lang1=en,lang2=as


ValueError: ignored

We see that the data is quite large - it has over a million translations. Since it's only text data, it should be relatively easy to process though.

Next, let's look at how the data was preprocessed when training the fine-tuned *XLS-R* checkpoint in Swedish. Looking at the [`run.sh` file](https://huggingface.co/hf-test/xls-r-300m-sv/blob/main/run.sh), we can see that the following characters were removed from the official transcriptions:

In [ ]:
chars_to_remove_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\']'  # change to the ignored characters of your fine-tuned model

Let's do the same here so that the alphabet of our language model matches the one of the fine-tuned acoustic checkpoints.

We can write a single map function to extract the Swedish text and process it right away.

In [ ]:
import re

def extract_text(batch):
  text = batch["translation"][target_lang]
  batch["text"] = re.sub(chars_to_ignore_regex, "", text.lower())
  return batch

Let's apply the `.map()` function. This should take roughly 5 minutes.

In [ ]:
dataset = dataset.map(extract_text, remove_columns=dataset.column_names)
dataset

  0%|          | 0/281 [00:00<?, ?ex/s]

KeyError: ignored

Great. Our dataset is already finished. Let's upload it to the Hub so that we can inspect and reuse it better.

You can log in by executing the following cell.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


Next, we call 🤗 Hugging Face's [`push_to_hub`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=push#datasets.Dataset.push_to_hub) method to upload the dataset to the repo `"swedish_corpora_parliament_processed"`.

In [ ]:
dataset.push_to_hub(f"{target_lang}_corpora_parliament_processed", split="train")

The repository already exists: the `private` keyword argument will be ignored.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

That was easy! The dataset viewer is automatically enabled when uploading a new dataset, which is very convenient. You can now directly inspect the dataset online. 

Feel free to look through our preprocessed dataset directly on [`hf-test/sv_corpora_parliament_processed`](https://huggingface.co/datasets/hf-test/sv_corpora_parliament_processed). Even if we are not a native speaker in Swedish, we can see that the data is well processed and seems clean.

Next, let's use the data to build a language model.

## **3. Build an *n-gram* with KenLM**

While large language models based on the [Transformer architecture](https://jalammar.github.io/illustrated-transformer/) have become the standard in NLP, it is still very common to use an ***n-gram*** LM to boost speech recognition systems - as shown in Section 1.

Looking again at Table 9 of Appendix C of the [official Wav2Vec2 paper](https://arxiv.org/abs/2006.11477), it can be noticed that using a *Transformer*-based LM for decoding clearly yields better results than using an *n-gram* model, but the difference between *n-gram* and *Transformer*-based LM is much less significant than the difference between *n-gram* and no LM. 

*E.g.*, for the large Wav2Vec2 checkpoint that was fine-tuned on 10min only, an *n-gram* reduces the word error rate (WER) compared to no LM by *ca.* 80% while a *Transformer*-based LM *only* reduces the WER by another 23% compared to the *n-gram*. This relative WER reduction becomes less, the more data the acoustic model has been trained on. *E.g.*, for the large checkpoint a *Transformer*-based LM reduces the WER by merely 8% compared to an *n-gram* LM whereas the *n-gram* still yields a 21% WER reduction compared to no language model.

The reason why an *n-gram* is preferred over a *Transformer*-based LM is that *n-grams* come at a significantly smaller computational cost. For an *n-gram*, retrieving the probability of a word given previous words is almost only as computationally expensive as querying a look-up table or tree-like data storage - *i.e.* it's very fast compared to modern *Transformer*-based language models that would require a full forward pass to retrieve the next word probabilities.

For more information on how *n-grams* function and why they are (still) so useful for speech recognition, the reader is advised to take a look at [this excellent summary](https://web.stanford.edu/~jurafsky/slp3/3.pdf) from Stanford.

Great, let's see step-by-step how to build an *n-gram*. We will use the popular [KenLM library](https://github.com/kpu/kenlm) to do so. Let's start by installing the Ubuntu library prerequisites:

In [ ]:
!sudo apt install build-essential cmake libboost-system-dev libboost-thread-dev libboost-program-options-dev libboost-test-dev libeigen3-dev zlib1g-dev libbz2-dev liblzma-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
libboost-program-options-dev is already the newest version (1.65.1.0ubuntu1).
libboost-program-options-dev set to manually installed.
libboost-system-dev is already the newest version (1.65.1.0ubuntu1).
libboost-system-dev set to manually installed.
libboost-thread-dev is already the newest version (1.65.1.0ubuntu1).
libboost-thread-dev set to manually installed.
liblzma-dev is already the newest version (5.2.2-1.3).
liblzma-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libboost-test-dev is already the newest version (1.65.1.0ubuntu1).
libboost-test-dev set to manually installed.
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
libbz2-dev is already the newest version (1.0.6-8.1ubuntu0.2).
libbz2-dev set to manually installed.
Su

before downloading and unpacking the KenLM repo.

In [ ]:
!wget -O - https://kheafield.com/code/kenlm.tar.gz | tar xz

--2022-01-23 10:17:35--  https://kheafield.com/code/kenlm.tar.gz
Resolving kheafield.com (kheafield.com)... 35.196.63.85
Connecting to kheafield.com (kheafield.com)|35.196.63.85|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 491090 (480K) [application/x-gzip]
Saving to: ‘STDOUT’

-                   100%[===================>] 479.58K  --.-KB/s    in 0.06s   

2022-01-23 10:17:35 (7.97 MB/s) - written to stdout [491090/491090]



KenLM is written in C++, so we'll make use of `cmake` to build the binaries.

In [ ]:
!mkdir kenlm/build && cd kenlm/build && cmake .. && make -j2
!ls kenlm/build/bin

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Boost version: 1.65.1
-- Found the following Boost libraries:


Great, as we can see, the executable functions have successfully been built under `kenlm/build/bin/`.

KenLM by default computes an *n-gram* with [Kneser-Ney smooting](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing). All text data used to create the *n-gram* is expected to be stored in a text file.
We download our dataset and save it as a `.txt` file.

In [ ]:
# from datasets import load_dataset

# username = "hf-test"  # change to your username

# dataset = load_dataset(f"{username}/{target_lang}_corpora_parliament_processed", split="train")

# with open("text.txt", "w") as file:
#   file.write(" ".join(dataset["text"]))

reference https://indicnlp.ai4bharat.org/corpora/

In [ ]:
!wget https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/indiccorp/as.tar.xz

--2022-01-23 10:20:04--  https://storage.googleapis.com/ai4bharat-public-indic-nlp-corpora/indiccorp/as.tar.xz
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.45.112, 172.253.115.128, 172.253.63.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.45.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 66709392 (64M) [application/x-tar]
Saving to: ‘as.tar.xz’

as.tar.xz           100%[===================>]  63.62M  50.8MB/s    in 1.3s    

2022-01-23 10:20:06 (50.8 MB/s) - ‘as.tar.xz’ saved [66709392/66709392]



In [ ]:
!tar -xf as.tar.xz

In [ ]:
!head ./data/as/as.txt

NE INDIA: হোজাই জিলাৰ লংকাৰ কাকীত বুধবাৰে নিশা স্থানীয় ৰাইজে এটা উট সহ দুজন ব্যক্তিক আটক কৰি গতাই দিয়ে কাকী আৰক্ষীক | আৰক্ষীয়ে আটকাধীন ব্যক্তি দুজনৰ ওপৰত তদন্ত অব্যাহত ৰাখিছে।
ইয়াৰে প্ৰায় ১৫জন যাত্ৰী আহত হৈছে।
প্রতিটো স্তৰত ৬টা কৈ সৰু ফুল ফুলে ।
উল্লেখ্য যে অহাবছৰ এই কেন্দ্ৰটোৰ উদ্যোগতে গুৱাহাটীত ৰাষ্ট্ৰীয় জুনিয়ৰ ইকুৱেষ্ট্ৰিয়ান প্ৰতিযোগিতৰো আয়োজনৰ পৰিকল্পনা হাতত লোৱা হৈছে।
Home > কহঁৰা মধ্য ইংৰাজী বিদ্যালয়ৰ ছাত্ৰৰ মৃত্যু
সাংবাদিকে বুঢ়া পাগল হোৱা বুলি আঁতৰি গ’ল।
কাৰণ যিকোনো এটা জনগোষ্ঠীয়ে তেওঁলোকৰ জনসংখ্যাৰ ভিত্তিত ভাষা ব্যৱহাৰৰ দাবী কৰিব পাৰে৷ এই দিশত আমি কেৱল ব্রহ্মপুত্র উপত্যকাৰ কথা ভাবিলেই নহ’ব৷ ভাবিব লাগিব বৰাক উপত্যকাৰ কথাও৷ সংখ্যাগত দিশৰ কথা আহিলেই সেই লোকসকলে বৰাক উপত্যকাকো সামৰি ল’ব৷ যদিহে তেনেকুৱা পৰিস্থিতিৰ সৃষ্টি হয় তেন্তে অসমত বাংলাভাষীৰ সংখ্যা কিমান হ’বগৈ?
‘কিন্তু তাৰ পিছতো এয়া কোনেও লক্ষ্য কৰা নাই!
দেশত ভাল শিক্ষকৰ সাংঘাতিক অভাৱ হৈছে৷ সাম্প্রতিক সময়ত নতুন শিক্ষানীতিৰ সমান্তৰালভাৱে ভাল শিক্ষক গঢ়াটোও গুৰুত্বপূর্ণ হৈ পৰিছে৷– এই মন্তব্য ডিব্রুগড় বিশ্ববিদ্যালয়ৰ প্রাক্তন উপাচ

In [ ]:
%%time
!sed 's/[a-zA-Z0-9>:]//g' ./data/as/as.txt > as_cleaned.txt
!sed 's/।/ । /g' ./as_cleaned.txt > as_cleaned.1.txt
!mv as_cleaned.1.txt text.txt
!ls -ltr

total 1080944
-rw-r--r-- 1 root root  66709392 Sep 23  2020 as.tar.xz
drwxr-xr-x 1 root root      4096 Jan  7 14:33 sample_data
drwxr-xr-x 2 root root      4096 Jan 23 09:15 cv-corpus-7.0-2021-07-21
drwxr-xr-x 9 1000 1000      4096 Jan 23 10:18 kenlm
drwxr-xr-x 3 root root      4096 Jan 23 10:21 data
-rw-r--r-- 1 root root 519100593 Jan 23 10:29 as_cleaned.txt
-rw-r--r-- 1 root root 521052779 Jan 23 10:29 text.txt
CPU times: user 3.21 s, sys: 680 ms, total: 3.89 s
Wall time: 7min 5s


Now, we just have to run KenLM's `lmplz` command to build our *n-gram*, called `"5gram.arpa"`. As it's relatively common in speech recognition, we build a *5-gram* by passing the `-o 5` parameter.
For more information on the different *n-gram* LM that can be built 
with KenLM, one can take a look at the [official website of KenLM](https://kheafield.com/code/kenlm/).

Executing the command below might take a minute or so.

In [ ]:

!kenlm/build/bin/lmplz -o 5 <"text.txt" > "5gram.arpa"

=== 1/5 Counting and sorting n-grams ===
Reading /content/text.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 1918697472 bytes == 0x55fd88f08000 @  0x7fa8c06f61e7 0x55fd883987a2 0x55fd8833351e 0x55fd883122eb 0x55fd882fe066 0x7fa8be88fbf7 0x55fd882ffbaa
tcmalloc: large alloc 8953896960 bytes == 0x55fdfb4d8000 @  0x7fa8c06f61e7 0x55fd883987a2 0x55fd883877ca 0x55fd88388208 0x55fd88312308 0x55fd882fe066 0x7fa8be88fbf7 0x55fd882ffbaa
****************************************************************************************************
Unigram tokens 30408490 types 1100270
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:13203240 2:1061907136 3:1991075968 4:3185721344 5:4645843968
tcmalloc: large alloc 4645847040 bytes == 0x55fd88f08000 @  0x7fa8c06f61e7 0x55fd883987a2 0x55fd883877ca 0x55fd88388208 0x55fd883128d7 0x55fd882fe066 0x7fa8be88fbf7 0x55fd882ffbaa
tcmalloc: large alloc 1991081984 bytes =

Great, we have built a *5-gram* LM! Let's inspect the first couple of lines.

In [ ]:
!head -20 5gram.arpa

\data\
ngram 1=1100270
ngram 2=10196386
ngram 3=20202858
ngram 4=24388635
ngram 5=25279719

\1-grams:
-7.0538974	<unk>	0
0	<s>	-1.0157648
-2.1170783	</s>	0
-4.0849	হোজাই	-0.7007276
-3.6245263	জিলাৰ	-0.3177578
-4.660954	লংকাৰ	-0.24585101
-6.026935	কাকীত	-0.11453828
-3.6779141	বুধবাৰে	-0.4127841
-3.7027547	নিশা	-0.3863523
-3.4385362	স্থানীয়	-0.5747764
-3.4661708	ৰাইজে	-0.4619786
-2.8084285	এটা	-0.53903985


There is a small problem that 🤗 Transformers will not be happy about later on.
The *5-gram* correctly includes a "Unknown" or `<unk>`, as well as a *begin-of-sentence*, `<s>` token, but no *end-of-sentence*, `</s>` token.
This sadly has to be corrected currently after the build.

We can simply add the *end-of-sentence* token by adding the line `0 </s>  -0.11831701` below the *begin-of-sentence* token and increasing the `ngram 1` count by 1. Because the file has roughly 100 million lines, this command will take *ca.* 2 minutes.

In [ ]:
!wc -l 5gram.arpa

81167886 5gram.arpa


In [ ]:
from tqdm import tqdm

with open("5gram.arpa", "r") as read_file, open("5gram_correct.arpa", "w") as write_file:
  has_added_eos = False
  for line in tqdm(read_file, total=81167886):
    if not has_added_eos and "ngram 1=" in line:
      count=line.strip().split("=")[-1]
      write_file.write(line.replace(f"{count}", f"{int(count)+1}"))
    elif not has_added_eos and "<s>" in line:
      write_file.write(line)
      write_file.write(line.replace("<s>", "</s>"))
      has_added_eos = True
    else:
      write_file.write(line)

100%|██████████| 81167886/81167886 [02:02<00:00, 663949.54it/s]


Let's now inspect the corrected *5-gram*.

In [ ]:
!head -20 5gram_correct.arpa

\data\
ngram 1=1100271
ngram 2=10196386
ngram 3=20202858
ngram 4=24388635
ngram 5=25279719

\1-grams:
-7.0538974	<unk>	0
0	<s>	-1.0157648
0	</s>	-1.0157648
-2.1170783	</s>	0
-4.0849	হোজাই	-0.7007276
-3.6245263	জিলাৰ	-0.3177578
-4.660954	লংকাৰ	-0.24585101
-6.026935	কাকীত	-0.11453828
-3.6779141	বুধবাৰে	-0.4127841
-3.7027547	নিশা	-0.3863523
-3.4385362	স্থানীয়	-0.5747764
-3.4661708	ৰাইজে	-0.4619786


Great, this looks better! We're done at this point and all that is left to do is to correctly integrate the `"ngram"` with [`pyctcdecode`](https://github.com/kensho-technologies/pyctcdecode) and 🤗 Transformers.

## **4. Combine an *n-gram* with Wav2Vec2**

In a final step, we want to wrap the *5-gram* into a `Wav2Vec2ProcessorWithLM` object to make the *5-gram* boosted decoding as seamless as shown in Section 1.
We start by downloading the currently "LM-less" processor of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv).

In [ ]:
repo_name

'infinitejoy/wav2vec2-large-xls-r-300m-assamese'

In [ ]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained(repo_name)

Next, we extract the vocabulary of its tokenizer as it represents the `"labels"` of `pyctcdecode`'s `BeamSearchDecoder` class.

In [ ]:
vocab_dict = processor.tokenizer.get_vocab()
sorted_vocab_dict = {k.lower(): v for k, v in sorted(vocab_dict.items(), key=lambda item: item[1])}

The `"labels"` and the previously built `5gram_correct.arpa` file is all that's needed to build the decoder. 

In [ ]:
ls -lltr

total 14710016
-rw-r--r-- 1 root root   66709392 Sep 23  2020 as.tar.xz
drwxr-xr-x 1 root root       4096 Jan  7 14:33 sample_data/
drwxr-xr-x 2 root root       4096 Jan 23 09:15 cv-corpus-7.0-2021-07-21/
drwxr-xr-x 9 1000 1000       4096 Jan 23 10:18 kenlm/
drwxr-xr-x 3 root root       4096 Jan 23 10:21 data/
-rw-r--r-- 1 root root  519100593 Jan 23 10:29 as_cleaned.txt
-rw-r--r-- 1 root root  521052779 Jan 23 10:29 text.txt
-rw-r--r-- 1 root root 6978078557 Jan 23 10:34 5gram.arpa
-rw-r--r-- 1 root root 6978078575 Jan 23 10:38 5gram_correct.arpa


In [ ]:
# !wget https://huggingface.co/infinitejoy/wav2vec2-large-xls-r-300m-assamese/resolve/main/language_model/5gram_correct.arpa

In [ ]:
# !mv 5gram_correct.arpa.1 5gram_correct.arpa

In [ ]:
# !head 5gram_correct.arpa

In [ ]:
from pyctcdecode import build_ctcdecoder

decoder = build_ctcdecoder(
    labels=list(sorted_vocab_dict.keys()),
    kenlm_model_path="5gram_correct.arpa",
)

Found entries of length > 1 in alphabet. This is unusual unless style is BPE, but the alphabet was not recognized as BPE type. Is this correct?
Unigrams and labels don't seem to agree.


We can safely ignore the warning and all that is left to do now is to wrap the just created `decoder`, together with the processor's `tokenizer` and `feature_extractor` into a `Wav2Vec2ProcessorWithLM` class.

In [ ]:
from transformers import Wav2Vec2ProcessorWithLM

processor_with_lm = Wav2Vec2ProcessorWithLM(
    feature_extractor=processor.feature_extractor,
    tokenizer=processor.tokenizer,
    decoder=decoder
)

We want to directly upload the LM-boosted processor into 
the model folder of [`xls-r-300m-sv`](https://huggingface.co/hf-test/xls-r-300m-sv) to have all relevant files in one place.

Let's clone the repo, add the new decoder files and upload them afterward.
First, we need to install `git-lfs`.

In [ ]:
!sudo apt-get install git-lfs tree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs tree
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,169 kB of archives.
After this operation, 7,767 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 2,169 kB in 1s (2,684 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Select

Cloning and uploading of modeling files can be done conveniently with the `huggingface_hub`'s `Repository` class. 

More information on how to use the `huggingface_hub` to upload any files, please take a look at the [official docs](https://huggingface.co/docs/hub/how-to-upstream).

In [ ]:
print(repo_name)

infinitejoy/wav2vec2-large-xls-r-300m-assamese


In [ ]:
from huggingface_hub import Repository

repo = Repository(local_dir=repo_name, clone_from=repo_name)

Cloning https://huggingface.co/infinitejoy/wav2vec2-large-xls-r-300m-assamese into local empty directory.


Download file language_model/5gram_correct.arpa:   0%|          | 1.58k/6.50G [00:00<?, ?B/s]

Download file runs/Jan19_14-02-34_Yellow/1642581173.3815742/events.out.tfevents.1642581173.Yellow.1965976.1: 1…

Download file runs/Jan18_11-45-41_Yellow/events.out.tfevents.1642486556.Yellow.1854158.6: 100%|##########| 7.2…

Download file runs/Jan19_14-02-34_Yellow/events.out.tfevents.1642581173.Yellow.1965976.0:  68%|######7   | 3.4…

...

Clean file runs/Jan18_10-29-03_Yellow/events.out.tfevents.1642481962.Yellow.1854158.0:  24%|##4       | 1.00k/…

Clean file training_args.bin:  34%|###3      | 1.00k/2.98k [00:00<?, ?B/s]

Clean file language_model/unigrams.txt:   0%|          | 1.00k/27.4M [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

Clean file language_model/5gram_correct.arpa:   0%|          | 1.00k/6.50G [00:00<?, ?B/s]

Having cloned `xls-r-300m-sv`, let's save the new processor with LM into it.

In [ ]:
!rm -rf infinitejoy/wav2vec2-large-xls-r-300m-assamese/language_model

In [ ]:
processor_with_lm.save_pretrained(repo_name)

Let's inspect the local repository. The `tree` command conveniently can also show the size of the different files.

In [ ]:
!tree -h infinitejoy/wav2vec2-large-xls-r-300m-assamese

infinitejoy/wav2vec2-large-xls-r-300m-assamese
├── [  23]  added_tokens.json
├── [ 703]  alphabet.json
├── [2.0K]  config.json
├── [4.0K]  language_model
│   ├── [6.5G]  5gram_correct.arpa
│   ├── [  78]  attrs.json
│   └── [ 27M]  unigrams.txt
├── [1.2K]  log_mozilla-foundation_common_voice_7_0_as_test_predictions.txt
├── [1.3K]  log_mozilla-foundation_common_voice_7_0_as_test_targets.txt
├── [  48]  mozilla-foundation_common_voice_7_0_as_test_eval_results.txt
├── [ 262]  preprocessor_config.json
├── [1.2G]  pytorch_model.bin
├── [2.3K]  README.md
├── [4.0K]  runs
│   ├── [4.0K]  Jan18_10-29-03_Yellow
│   │   ├── [4.0K]  1642481962.2863166
│   │   │   └── [4.5K]  events.out.tfevents.1642481962.Yellow.1854158.1
│   │   └── [4.1K]  events.out.tfevents.1642481962.Yellow.1854158.0
│   ├── [4.0K]  Jan18_10-33-31_Yellow
│   │   ├── [4.0K]  1642482228.516824
│   │   │   └── [4.5K]  events.out.tfevents.1642482228.Yellow.1854158.3
│   │   └── [5.9K]  events.out.tfevents.1642482228.Yellow.18541

As can be seen the *5-gram* LM is quite large - it amounts to more than 4 GB.
To reduce the size of the *n-gram* and make loading faster, `kenLM` allows converting `.arpa` files to binary ones using the `build_binary` executable.

Let's make use of it here.

In [ ]:
!kenlm/build/bin/build_binary infinitejoy/wav2vec2-large-xls-r-300m-assamese/language_model/5gram_correct.arpa xls-r-300m-sv/language_model/5gram.bin

Reading infinitejoy/wav2vec2-large-xls-r-300m-assamese/language_model/5gram_correct.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
/content/kenlm/util/file.cc:86 in int util::CreateOrThrow(const char*) threw ErrnoException because `-1 == (ret = open(name, 0100 | 01000 | 02, 0400 | 0200 | (0400 >> 3) | ((0400 >> 3) >> 3)))'.
No such file or directory while creating xls-r-300m-sv/language_model/5gram.bin Byte: 92
ERROR


Great, it worked! Let's remove the `.arpa` file and check the size of the binary *5-gram* LM.

In [ ]:
!ls -ltr infinitejoy/wav2vec2-large-xls-r-300m-assamese/language_model/

total 28060
-rw-r--r-- 1 root root       78 Jan 23 12:36 attrs.json
-rw-r--r-- 1 root root 28727688 Jan 23 12:36 unigrams.txt


In [ ]:
# !rm infinitejoy/wav2vec2-large-xls-r-300m-assamese/language_model/5gram_correct.arpa && tree -h xls-r-300m-sv/

xls-r-300m-sv/ [error opening dir]

0 directories, 0 files


Nice, we reduced the *n-gram* by more than half to less than 2GB now. In the final step, let's upload all files.

In [ ]:
repo.push_to_hub(commit_message="Upload lm-boosted decoder")

To https://huggingface.co/infinitejoy/wav2vec2-large-xls-r-300m-assamese
   1138267..fc7e06c  main -> main



'https://huggingface.co/infinitejoy/wav2vec2-large-xls-r-300m-assamese/commit/fc7e06ce3d8061e071a4ba47f82d393660386fd4'

That's it. Now you should be able to use the *5gram* for LM-boosted decoding as shown in Section 1.

As can be seen on [`xls-r-300m-sv`'s model card](https://huggingface.co/hf-test/xls-r-300m-sv#inference-with-lm) our *5gram* LM-boosted decoder yields a WER of 18.85% on Common Voice's 7 test set which is a relative performance of *ca.* 30% 🔥.

In [ ]:
# from datasets import Audio, Dataset, load_dataset, load_metric
# from transformers import AutoFeatureExtractor, pipeline

# dataset = load_dataset("mozilla-foundation/common_voice_7_0", "hi", split="test", use_auth_token=True)

# # for testing: only process the first two examples as a test
# dataset = dataset.select(range(10))

# # load processor
# feature_extractor = AutoFeatureExtractor.from_pretrained("shivam/xls-r-hindi")
# sampling_rate = feature_extractor.sampling_rate

# # resample audio
# dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

# # load eval pipeline
# asr = pipeline("automatic-speech-recognition", model="shivam/xls-r-hindi")

# # map function to decode audio
# def map_to_pred(batch):
#     prediction = asr(
#         batch["audio"]["array"])

#     batch["prediction"] = prediction["text"]
#     batch["target"] = batch["sentence"]
#     return batch

# # run inference on all examples
# result = dataset.map(map_to_pred, remove_columns=dataset.column_names)
# print(result["prediction"])

In [ ]:
repo_name

'infinitejoy/wav2vec2-large-xls-r-300m-assamese'

In [ ]:
# !pip install -U --force-reinstall transformers

     |████████████████████████████████| 3.4 MB 30.6 MB/s 
  Using cached huggingface_hub-0.4.0-py3-none-any.whl (67 kB)
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 76 kB 5.2 MB/s 
     |████████████████████████████████| 15.7 MB 31.4 MB/s 
     |████████████████████████████████| 748 kB 51.8 MB/s 
     |████████████████████████████████| 3.3 MB 41.9 MB/s 
     |████████████████████████████████| 40 kB 5.7 MB/s 
  Using cached sacremoses-0.0.47-py2.py3-none-any.whl (895 kB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
     |████████████████████████████████| 98 kB 7.7 MB/s 
     |████████████████████████████████| 138 kB 48.3 MB/s 
     |████████████████████████████████| 149 kB 56.0 MB/s 
     |████████████████████████████████| 61 kB 8.2 MB/s 
     |████████████████████████████████| 306 kB 56.7 MB/s 
     |████████████████████████████████| 97 kB 7.0 MB/s 


In [ ]:
!ls -ltr infinitejoy/wav2vec2-large-xls-r-300m-assamese

total 1232684
-rw-r--r-- 1 root root       2322 Jan 23 11:40 README.md
-rw-r--r-- 1 root root       2067 Jan 23 11:40 config.json
-rw-r--r-- 1 root root         48 Jan 23 11:40 mozilla-foundation_common_voice_7_0_as_test_eval_results.txt
-rw-r--r-- 1 root root       1376 Jan 23 11:40 log_mozilla-foundation_common_voice_7_0_as_test_targets.txt
-rw-r--r-- 1 root root       1270 Jan 23 11:40 log_mozilla-foundation_common_voice_7_0_as_test_predictions.txt
drwxr-xr-x 8 root root       4096 Jan 23 11:40 runs
-rw-r--r-- 1 root root       3055 Jan 23 11:40 training_args.bin
-rw-r--r-- 1 root root 1262206577 Jan 23 11:50 pytorch_model.bin
-rw-r--r-- 1 root root        262 Jan 23 12:36 preprocessor_config.json
-rw-r--r-- 1 root root        350 Jan 23 12:36 tokenizer_config.json
-rw-r--r-- 1 root root        695 Jan 23 12:36 special_tokens_map.json
-rw-r--r-- 1 root root        729 Jan 23 12:36 vocab.json
-rw-r--r-- 1 root root        703 Jan 23 12:36 alphabet.json
-rw-r--r-- 1 root root         

In [ ]:
from datasets import Audio, Dataset, load_dataset, load_metric
from transformers import AutoFeatureExtractor, pipeline

dataset = load_dataset("mozilla-foundation/common_voice_7_0", "as", split="test", use_auth_token=True)

# for testing: only process the first two examples as a test
dataset = dataset.select(range(10))

repo_name = 'infinitejoy/wav2vec2-large-xls-r-300m-assamese'

# load processor
feature_extractor = AutoFeatureExtractor.from_pretrained(repo_name)
# feature_extractor = processor_with_lm.feature_extractor
sampling_rate = feature_extractor.sampling_rate

# resample audio
dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

# load eval pipeline
asr = pipeline("automatic-speech-recognition", model=repo_name, feature_extractor=feature_extractor)

# map function to decode audio
def map_to_pred(batch):
    prediction = asr(
        batch["audio"]["array"])

    batch["prediction"] = prediction["text"]
    batch["target"] = batch["sentence"]
    return batch

# run inference on all examples
result = dataset.map(map_to_pred, remove_columns=dataset.column_names)
print(result["prediction"])

Reusing dataset common_voice (/root/.cache/huggingface/datasets/mozilla-foundation___common_voice/as/7.0.0/33e08856cfa0d0665e837bcad73ffd920a0bc713ce8c5fffb55dbdf1c084d5ba)


  0%|          | 0/10 [00:00<?, ?ex/s]

['যাৰাযততেউ তেজ্কৰলৈ যব কিনলতু যাহাযিতো আহিপনে।', 'কিলৰ গৰশ্ন শুনিকযেতিযকজনে উত্তৰ দিলে।', 'আৰুতেউৰা গৰ পুৱা নাই যেউল পাহৰিছিল।', 'কাণা ৰজাৰ গৈ আঠোলৈ কলে।', 'বাইকৰ ছোৰো মৰ লগতে আছে দৰ নমম ৰুত দুকনখনসি ঘনৰ গথা গৈছিল।', 'ডিকত কোৱা নাই মোৰা হছফুল।', 'মাষ্তুৰ কথা শুনি আনবোৰ মাছচ বগলীৰ লগত যাবলৈ খাজুহল।', 'এতিয়়াৰ মই টোমাকক ছচযিেজ আৰু ভাল ভাল পৃঠানিদিয়ঁ।', 'উননয্হতে দাত্জিনে দ্তত বিতি্ছৰ সফৈললৈ বোম্বাই সকুলো প্ৰকাৰ দেখিয় সক্তিৰ আক্ৰমনৰ হআাতৰ পৰা মখ্তি দিযে।', 'মই ভায় ঘদগ নিছিনজ দিও মন্দলঁচিনিক।']


In [ ]:
dataset

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 10
})

In [ ]:
import IPython.display as ipd

index = 3
audio_sample = dataset[index]
print(audio_sample["sentence"].lower())
ipd.Audio(data=audio_sample["audio"]["array"], autoplay=True, rate=audio_sample["audio"]["sampling_rate"])

In [ ]:
import IPython.display as ipd

index = 3
audio_sample = dataset[index]
print(audio_sample["sentence"].lower())
ipd.Audio(data=audio_sample["audio"]["array"], autoplay=True, rate=audio_sample["audio"]["sampling_rate"])

"কানায়ে ৰজাৰ আগত আঁঠু লৈ ক'লে।"


In [ ]:
result[3]

{'prediction': 'কাণা ৰজাৰ গৈ আঠোলৈ কলে।',
 'target': '"কানায়ে ৰজাৰ আগত আঁঠু লৈ ক\'লে।"'}

In [ ]:
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "processor_class": "Wav2Vec2ProcessorWithLM",
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [ ]:
processor_with_lm.feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [ ]:
for i, _ in enumerate(result):
    print(result[i])

{'prediction': 'যাৰাযততেউ তেজ্কৰলৈ যব কিনলতু যাহাযিতো আহিপনে।', 'target': 'জাহাজত তেওঁ তেজপুৰলৈ যাব, কিন্তু জাহাজ এতিয়াও আহি পোৱা নাই।'}
{'prediction': 'কিলৰ গৰশ্ন শুনিকযেতিযকজনে উত্তৰ দিলে।', 'target': 'উকিলৰ প্ৰশ্ন শুনি খেতিয়ক জনে উত্তৰ দিলে'}
{'prediction': 'আৰুতেউৰা গৰ পুৱা নাই যেউল পাহৰিছিল।', 'target': '"আৰু তেওঁৰ আগৰ পৰা নাইকিয়া হ\'লে পাহৰিছিল।"'}
{'prediction': 'কাণা ৰজাৰ গৈ আঠোলৈ কলে।', 'target': '"কানায়ে ৰজাৰ আগত আঁঠু লৈ ক\'লে।"'}
{'prediction': 'বাইকৰ ছোৰো মৰ লগতে আছে দৰ নমম ৰুত দুকনখনসি ঘনৰ গথা গৈছিল।', 'target': '"বাইকৰ শ্ব\'ৰুমৰ লগতে যে আছে ""দৰৱ"" নামৰ ঔষধৰ দোকানখন, সেইখনৰ কথা কৈছোঁ।"'}
{'prediction': 'ডিকত কোৱা নাই মোৰা হছফুল।', 'target': 'টিকেট পোৱা নাই, পূৰা হাউচফুল।'}
{'prediction': 'মাষ্তুৰ কথা শুনি আনবোৰ মাছচ বগলীৰ লগত যাবলৈ খাজুহল।', 'target': '"মাছটোৰ কথা শুনি আনবোৰ মাছ বগলীৰ লগত যাবলৈ সাজু হ\'ল।"'}
{'prediction': 'এতিয়়াৰ মই টোমাকক ছচযিেজ আৰু ভাল ভাল পৃঠানিদিয়ঁ।', 'target': '"এতিয়াই মই তোমাক ছ\'ছেজ আৰু ভাল ভাল পিঠা আনি দিওঁ।"'}
{'prediction': 'উননয্হতে দাত্জিন

In [ ]:
for i, _ in enumerate(result):
    print(result[i])

{'prediction': 'যাৰাযততেউ তেজ্কৰলৈ যব কিনলতু যাহাযিতো আহিপনে।', 'target': 'জাহাজত তেওঁ তেজপুৰলৈ যাব, কিন্তু জাহাজ এতিয়াও আহি পোৱা নাই।'}
{'prediction': 'কিলৰ গৰশ্ন শুনিকযেতিযকজনে উত্তৰ দিলে।', 'target': 'উকিলৰ প্ৰশ্ন শুনি খেতিয়ক জনে উত্তৰ দিলে'}
{'prediction': 'আৰুতেউৰা গৰ পুৱা নাই যেউল পাহৰিছিল।', 'target': '"আৰু তেওঁৰ আগৰ পৰা নাইকিয়া হ\'লে পাহৰিছিল।"'}
{'prediction': 'কাণা ৰজাৰ গৈ আঠোলৈ কলে।', 'target': '"কানায়ে ৰজাৰ আগত আঁঠু লৈ ক\'লে।"'}
{'prediction': 'বাইকৰ ছোৰো মৰ লগতে আছে দৰ নমম ৰুত দুকনখনসি ঘনৰ গথা গৈছিল।', 'target': '"বাইকৰ শ্ব\'ৰুমৰ লগতে যে আছে ""দৰৱ"" নামৰ ঔষধৰ দোকানখন, সেইখনৰ কথা কৈছোঁ।"'}
{'prediction': 'ডিকত কোৱা নাই মোৰা হছফুল।', 'target': 'টিকেট পোৱা নাই, পূৰা হাউচফুল।'}
{'prediction': 'মাষ্তুৰ কথা শুনি আনবোৰ মাছচ বগলীৰ লগত যাবলৈ খাজুহল।', 'target': '"মাছটোৰ কথা শুনি আনবোৰ মাছ বগলীৰ লগত যাবলৈ সাজু হ\'ল।"'}
{'prediction': 'এতিয়়াৰ মই টোমাকক ছচযিেজ আৰু ভাল ভাল পৃঠানিদিয়ঁ।', 'target': '"এতিয়াই মই তোমাক ছ\'ছেজ আৰু ভাল ভাল পিঠা আনি দিওঁ।"'}
{'prediction': 'উননয্হতে দাত্জিন